## 加载工具包

In [142]:
import time
import math
import numpy as np
from six.moves import urllib
import tensorflow as tf

## 读取数据

In [143]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 2002,
                                            class_mode = 'binary')

Found 8000 images belonging to 2 classes.
Found 2002 images belonging to 2 classes.


## 创建卷积网络

In [177]:
#创建权重变量
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.02)
  return tf.Variable(initial)
#创建偏差
def bias_variable(shape):
  initial = tf.constant(0.001, shape=shape)
  return tf.Variable(initial)

#卷积函数
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')
#池化函数
def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [178]:
graph = tf.Graph()
with graph.as_default():
    #变量定义
    #定义输入数据
    x = tf.placeholder(tf.float32, [None, 64, 64, 3])
    #期望输出标签
    y_ = tf.placeholder(tf.float32, [None,1])
    #丢弃因子
    keep_prob = tf.placeholder("float")

In [179]:
y_

<tf.Tensor 'Placeholder_1:0' shape=(?, 1) dtype=float32>

In [180]:
with graph.as_default():
    with tf.name_scope("conv1"):
        #第一层卷积
        W_conv1 = weight_variable([3, 3, 3, 32])
        b_conv1 = bias_variable([32])
        #卷积后采用Relu函数激活
    h_conv1 = tf.nn.relu(conv2d(x, W_conv1) + b_conv1)
    #进行池化
    h_pool1 = max_pool_2x2(h_conv1)
    with tf.name_scope("conv2"):
        #第二层卷积
        W_conv2 = weight_variable([3, 3, 32, 32])
        b_conv2 = bias_variable([32])
        #卷积后采用Relu函数激活
        h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
        #进行池化
        h_pool2 = max_pool_2x2(h_conv2)

In [181]:
W_conv1

In [182]:
h_pool2

<tf.Tensor 'conv2/MaxPool:0' shape=(?, 16, 16, 32) dtype=float32>

In [183]:
#全连接层
with graph.as_default():
    with tf.name_scope("fully_connected"):
        W_fc1 = weight_variable([16*16 * 32, 128])
        b_fc1 = bias_variable([128])
        h_pool2_flat = tf.reshape(h_pool2, [-1, 16*16*32])
        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [184]:
h_fc1

<tf.Tensor 'fully_connected/Relu:0' shape=(?, 128) dtype=float32>

In [185]:
#丢弃，为了增强泛化能力
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [186]:
#输出层，softmax
#全连接层
with graph.as_default():
    with tf.name_scope("output"):
        W_fc2 = weight_variable([128, 1])
        b_fc2 = bias_variable([1])
    y_conv=tf.nn.sigmoid(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [187]:
y_

<tf.Tensor 'Placeholder_1:0' shape=(?, 1) dtype=float32>

## 损失函数以及梯度下降算法

In [188]:
with graph.as_default():
    #损失函数
    cross_entropy = -tf.reduce_mean(y_*tf.log(y_conv) + (1-y_)*tf.log(1-y_conv))
    #梯度下降法，使用Adam算法
    optimizer = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
     #Decaying Learning Rate
    #cur_step = tf.Variable(0, trainable=False)  # count the number of steps taken.
    #starter_learning_rate = 0.001
    #learning_rate = tf.train.exponential_decay(starter_learning_rate, cur_step, 100000, 0.96, staircase=True)
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy, global_step=cur_step)
    #计算准确度
    preds = y_conv > 0.5
    correct_prediction = tf.equal(tf.cast(preds, tf.int32), tf.cast(y_, tf.int32))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [189]:
accuracy

<tf.Tensor 'Mean_1:0' shape=() dtype=float32>

In [190]:
num_steps = 501
epochs = 50
with tf.Session(graph=graph) as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    for _ in range(epochs):
        for step in range(num_steps):
            #产生训练用样本集      
            batch_data, batch_labels = training_set.next()
            batch_labels = np.reshape(batch_labels, [-1,1])
            #数据传递给tensorflow
            feed_dict = {x : batch_data, y_ : batch_labels, keep_prob:0.5}
            sess.run(optimizer, feed_dict=feed_dict)
            if step%100 == 0:
                #每50次计算准确度
                result = sess.run(cross_entropy, feed_dict={x : batch_data, y_ : batch_labels, keep_prob:1})
                print('Cost:', result)   
    print('Testing Result.....')       
    batch_data, batch_labels = test_set.next()
    batch_labels = np.reshape(batch_labels, [-1,1])
    feed_dict = {x : batch_data, y_ : batch_labels, keep_prob:1}
    result = sess.run(accuracy, feed_dict)
    print('Accuracy:', result)

Cost: 0.693007
Cost: 0.691502
Cost: 0.689338
Cost: 0.700539
Cost: 0.685761
Cost: 0.63397
Cost: 0.670364
Cost: 0.669262
Cost: 0.659475
Cost: 0.641019
Cost: 0.63135
Cost: 0.631915
Cost: 0.63762
Cost: 0.681392
Cost: 0.662226
Cost: 0.621458
Cost: 0.637389
Cost: 0.567379
Cost: 0.587076
Cost: 0.63213
Cost: 0.684681
Cost: 0.707753
Cost: 0.61392
Cost: 0.611486
Cost: 0.577305
Cost: 0.63776
Cost: 0.552176
Cost: 0.645295
Cost: 0.662904
Cost: 0.634555
Cost: 0.632221
Cost: 0.49478
Cost: 0.527905
Cost: 0.534481
Cost: 0.582598
Cost: 0.548433
Cost: 0.420304
Cost: 0.552564
Cost: 0.516261
Cost: 0.66945
Cost: 0.677477
Cost: 0.636271
Cost: 0.502658
Cost: 0.778908
Cost: 0.507724
Cost: 0.53839
Cost: 0.455044
Cost: 0.51519
Cost: 0.675639
Cost: 0.407274
Cost: 0.721305
Cost: 0.51435
Cost: 0.652092
Cost: 0.574896
Cost: 0.54661
Cost: 0.498424
Cost: 0.442943
Cost: 0.508728
Cost: 0.552254
Cost: 0.458525
Cost: 0.578653
Cost: 0.415252
Cost: 0.410941
Cost: 0.564578
Cost: 0.535981
Cost: 0.380072
Cost: 0.554747
Cost: 0